# EP Tender : simulation par chaînes de Markov

## Description du modèle

On considère une station comportant $f$ files, pour un total de $n$ tenders en circulation. Chaque tender comporte $d + 1$ états de charge, numérotés de 0 à $d$. À 0 un tender est complètement déchargé, et à $d$ il est complètement chargé.

On utilise un modèle à temps discret.

Les voitures clientes arrivent de sorte que l'écart temporel entre la $i$-ième voiture et la $(i+1)$-ième est de loi géométrique de paramètre $\lambda$. Lorsqu'elles arrivent elles doivent prendre un tender complètement chargé qu'elles ramènent complètement déchargé au début de l'étape suivante.

Fatalement, un jour plus de $n$ voitures arriveront en même temps et alors la demande ne pourra pas être satisfaite. On note $T_\mathrm{bug}$ le premier temps où la demande en Tender est insatisfaite et on s'intéresse à son espérance. On cherche à trouver, pour un processus de rangement et de sélection des tenders donné, l'espérance de $T_\mathrm{bug}$, qu'on souhaite maximal.

Chaque étape temporelle se déroule comme suit :
1. Tous les tenders dans la station gagnent un niveau de charge, pour un niveau maximum de $d$.
2. Les tenders partis à l'étape précédente sont rangés dans la station.
3. Les nouvelles voitures clientes arrivent et des tenders chargés leur sont attribués.
4. Si la demande a pu être satisfaite, on passe au temps suivant.

## Importation des modules

In [1]:
import numpy as np

## Encodage des états

### Encodage d'une file simple

Une file donnée contient un certain nombre de tenders, chacun à un état de charge donné. Remarquons que les tenders les plus au fond sont ceux arrivés le plus tôt, ils sont donc les plus chargés. On peut donc représenter une file par une suite décroissante d'entiers entre 0 et $d$, de longueur au plus $n$. Si $d < n$ il peut être plus efficace d'encoder cette suite par une suite de $d + 1$ entiers positifs, de somme au plus $n$, représentant le nombre de tenders de niveau de charge $d$ dans la file, puis le nombre de tenders de niveau de charge $d - 1$, et ainsi de suite jusque $0$.

In [17]:
def encode(file, d) :
# Prend une suite décroissante d'entiers positifs entre 0 et d, et renvoie le nombre d'entiers pour chaque valeur

    fileEncodee = np.zeros(d + 1, dtype = int)
    for charge in file :
        fileEncodee[d - charge] += 1
    
    return fileEncodee

In [18]:
file = [5, 5, 3, 2, 2, 1, 0]
fileEncodee = encode(file, 6)
fileEncodee

array([0, 2, 0, 1, 2, 1, 1])

In [19]:
def decode(fileEncodee) :
    # Prend une suite d'entiers positifs, et renvoie une suite d'entiers, de longueur la somme de la suite originale
    # et prenant chaque valeur le nombre de fois indiqué par la suite de départ, de sorte à se terminer par la valeur 0
    
    d = len(fileEncodee) - 1
    file = np.zeros(sum(fileEncodee), dtype = int)
    i = 0
    for v in fileEncodee :
        while v > 0 :
            file[i] = d
            i += 1
            v -= 1
        d -= 1
    
    return file

In [20]:
decode(fileEncodee)

array([5, 5, 3, 2, 2, 1, 0])

### Encodage d'une station

Une station comportant $f$ files peut être représentée par une matrice avec $f$ éléments, chaque élément listant les états des tenders dans une file. On complète les files par des -1 pour les ramener toutes à la même longueur.

In [74]:
def encodeStation(station, d) :
    # Prend une station et renvoie la matrice comportant la valeur encodée de chaque ligne
    
    if type(station) == np.ndarray :
        station = station.tolist()
    
    stationEncodee = []
    for indexFile in range(len(station)) :
        file = station[indexFile]
        try :
            finFile = file.index(-1)
        except ValueError :
            finFile = len(file)
        stationEncodee.append(encode(file[:finFile], d))
    
    return np.array(stationEncodee, dtype = int)

In [62]:
station = [[5, 5, 3, 2, 2, 1, 0], [3, 3]]
stationEncodee = encodeStation(station, 6)
stationEncodee

array([[0, 2, 0, 1, 2, 1, 1],
       [0, 0, 0, 2, 0, 0, 0]])

In [88]:
def padStation(station) :
    # Prend une station représentée sous forme de liste et la renvoie sous forme de np.array, complété de -1
    
    if type(station) == list :
        longueurFiles = max([len(file) for file in station])
        for indexFile, file in enumerate(station) :
            station[indexFile] = file + [-1] * (longueurFiles - len(file))
    
    return np.array(station)

In [89]:
padStation(station)

array([[ 5,  5,  3,  2,  2,  1,  0],
       [ 3,  3, -1, -1, -1, -1, -1]])

In [90]:
def decodeStation(stationEncodee) :
    # Prend une station encodée et renvoie la liste de ses files décodées
    
    if type(stationEncodee) == list :
        stationEncodee = np.array(stationEncodee)
    
    station = []
    
    for indexFile in range(stationEncodee.shape[0]) :
        fileEncodee = stationEncodee[indexFile, :]
        station.append(decode(fileEncodee).tolist())
        
    return padStation(station)

In [91]:
decodeStation(stationEncodee)

array([[ 5,  5,  3,  2,  2,  1,  0],
       [ 3,  3, -1, -1, -1, -1, -1]])

## Charge des tenders

Une même fonction permet de réaliser la charge des stations et des files. Il faut néanmoins différencier le cas encodé et décodé.

In [123]:
def charge(station, d) :
    # Prend une station (décodée) et la renvoie après que chaque tender ait été chargé de 1
    
    if type(station) == list :
        station = padStation(station)
        
    station += ~np.isin(station, [-1, d])    
    
    return station

In [126]:
station

[[5, 5, 3, 2, 2, 1, 0], [3, 3, -1, -1, -1, -1, -1]]

In [125]:
charge(station, 5)

array([[ 5,  5,  4,  3,  3,  2,  1],
       [ 4,  4, -1, -1, -1, -1, -1]])

In [133]:
def chargeEncodee(station) :
    # Prend une station (encodée) et la renvoie après que chaque tender ait été chargé de 1
    
    station = padStation(station)    
    chargedStation = np.zeros(station.shape)
    
    for indexFile in range(station.shape[0]) :
        chargedStation[indexFile, 0] = station[indexFile, 0]
        chargedStation[indexFile, :-1] += station[indexFile, 1:]
    
    return chargedStation

In [136]:
stationEncodee

array([[0, 2, 0, 1, 2, 1, 1],
       [0, 0, 0, 2, 0, 0, 0]])

In [137]:
chargeEncodee(stationEncodee)

array([[2., 0., 1., 2., 1., 1., 0.],
       [0., 0., 2., 0., 0., 0., 0.]])

In [138]:
chargeEncodee(chargeEncodee(chargeEncodee(stationEncodee)))

array([[3., 2., 1., 1., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0.]])

## Réduction du nombre de configurations pour les stations

Les états de notre chaîne de Markov sont les configurations de la station. D'un point de vue de la satisfaction de la demande, cependant